In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm


c:\Users\Jaime\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] No se encontró el proceso especificado'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
ruta_base = "clasificador_analisis/clasificador/datasets_originales/"
path = ruta_base + "politicos_manual_completo.xlsx"

df_posts = pd.read_excel(path, sheet_name="Posts")

## Tema

In [ ]:
etiqueta_map = {
    0: "Gestión Pública e Instituciones",
    1: "Economía, Empresa, Empleo e Infraestructuras",
    2: "Sociedad, Igualdad y Derechos",
    3: "Otros"
}

model_path = "clasificador_analisis/clasificador/clasificadores_finales/modelo_final_tema"

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval()

def predict_tema(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).squeeze()
    return int(torch.argmax(probs))

ruta_base = "clasificador_analisis/clasificador/datasets_originales/"
input_path = ruta_base + "politicos_manual_completo.xlsx"
xlsx = pd.ExcelFile(input_path)

df_posts = pd.read_excel(xlsx, sheet_name="Posts")
df_comentarios = pd.read_excel(xlsx, sheet_name="Comentarios")
df_metadata = pd.read_excel(xlsx, sheet_name="Metadata")

posts_sin_tema = df_posts[df_posts["Tema"].isna()].copy()

tqdm.pandas(desc="Clasificando temas")
posts_sin_tema["Tema"] = posts_sin_tema["Contenido_Traducido"].progress_apply(
    lambda x: etiqueta_map[predict_tema(str(x))] if pd.notnull(x) else None
)

df_posts.update(posts_sin_tema)

output_path = "clasificador_analisis/clasificador/clasificadores_finales/politicos_etiquetado_tema.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_metadata.to_excel(writer, index=False, sheet_name="Metadata")
    df_posts.to_excel(writer, index=False, sheet_name="Posts")
    df_comentarios.to_excel(writer, index=False, sheet_name="Comentarios")

print("✅ Clasificación de temas completada y guardada.")


Clasificando temas: 100%|██████████| 8597/8597 [07:31<00:00, 19.03it/s]


✅ Clasificación de temas completada y guardada.


## Tono

In [ ]:
model_path_tono = "clasificador_analisis/clasificador/clasificadores_finales/modelo_final_tono"

etiqueta_map_tono = {
    0: "Negativo",
    1: "Neutro",
    2: "Positivo"
}

tokenizer_tono = BertTokenizer.from_pretrained(model_path_tono)
model_tono = BertForSequenceClassification.from_pretrained(model_path_tono)
model_tono.eval()

def predict_tono(text):
    inputs = tokenizer_tono(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model_tono(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).squeeze()
    return int(torch.argmax(probs))

input_path = "clasificador_analisis/clasificador/clasificadores_finales/politicos_etiquetado_tema.xlsx"
xlsx = pd.ExcelFile(input_path)

df_posts = pd.read_excel(xlsx, sheet_name="Posts")
df_comentarios = pd.read_excel(xlsx, sheet_name="Comentarios")
df_metadata = pd.read_excel(xlsx, sheet_name="Metadata")

posts_sin_tono = df_posts[df_posts["Tono"].isna()].copy()
comentarios_sin_tono = df_comentarios[df_comentarios["Tono"].isna()].copy()

respuestas_validas = (
    df_comentarios["Respuesta_Traducida"].notna() &
    (df_comentarios["Respuesta_Traducida"].str.strip().str.lower() != "no")
)
respuestas_sin_tono = df_comentarios[respuestas_validas & df_comentarios["Tono_Respuesta"].isna()].copy()

tqdm.pandas(desc="Clasificando tono en Posts")
posts_sin_tono["Tono"] = posts_sin_tono["Contenido_Traducido"].progress_apply(
    lambda x: etiqueta_map_tono[predict_tono(str(x))] if pd.notnull(x) else None
)

tqdm.pandas(desc="Clasificando tono en Comentarios")
comentarios_sin_tono["Tono"] = comentarios_sin_tono["Comentario_Traducido"].progress_apply(
    lambda x: etiqueta_map_tono[predict_tono(str(x))] if pd.notnull(x) else None
)

tqdm.pandas(desc="Clasificando tono en Respuestas")
respuestas_sin_tono["Tono_Respuesta"] = respuestas_sin_tono["Respuesta_Traducida"].progress_apply(
    lambda x: etiqueta_map_tono[predict_tono(str(x))] if pd.notnull(x) else None
)

df_posts.update(posts_sin_tono)
df_comentarios.update(comentarios_sin_tono)
df_comentarios.update(respuestas_sin_tono)

output_path = "clasificador_analisis/analisis/datasets/politicos_etiquetado_completo.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_metadata.to_excel(writer, index=False, sheet_name="Metadata")
    df_posts.to_excel(writer, index=False, sheet_name="Posts")
    df_comentarios.to_excel(writer, index=False, sheet_name="Comentarios")

print("✅ Clasificación de tono completada y guardada.")


Clasificando tono en Respuestas: 100%|██████████| 200/200 [00:07<00:00, 25.99it/s]


✅ Clasificación de tono completada y guardada.
